In [1]:
import sys
sys.path.append('scripts/dialent/task1/')
sys.path.append('scripts/dialent/')
sys.path.append('scripts/')
import util

In [2]:
import numpy as np
import pandas as pd
import pymorphy2
import re, string

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import xgboost as xgb

In [4]:
u_dev = util.loadAllStandard('./devset/')
u_test = util.loadAllStandard('./testset//')

Failed to load the standard of book_3954:
Unknown mention tag: Facility


In [5]:
morph = pymorphy2.MorphAnalyzer()
def get_pos(token):
    pos = morph.parse(token)[0].tag.POS
    if pos == 'NOUN':
        return pos
    return "none"

In [6]:
def ne_bin(x):
    if x == 'LOC':
        return 0
    elif x == 'LOCORG':
        return 1

In [7]:
def pos_bin(x):
    if x == 'NOUN':
        return 1
    if x == 'ADJF':
        return 2
    if x == 'PRTF':
        return 3
    if x == 'VERB':
        return 4
    if x == 'CONJ':
        return 5
    return 0

In [8]:
def get_df(u):
    df = pd.DataFrame(columns=['tokens', 'tokens_id', 'start', 'end', 'ne'])
    for i in range(len(u)):
        res = u[i].makeTokenSets()
        for j in range(len(res)):
            token = res[j].toInlineString()
            split = token.split()
            df = df.append({'tokens': split[4:], 'first_token': str(split[4].split('"')[1]), 'tokens_id': int(split[1]), 'start': int(split[2][1:-1]), \
               'end': int(split[3][:-1]), 'ne': str(split[0])}, ignore_index=True)
    return df

In [9]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [10]:
df_dev = get_df(u_dev)
df_dev = df_dev[(df_dev['ne'] == 'LOC') | (df_dev['ne'] == 'LOCORG')]
df_dev['ne_bin'] = df_dev['ne'].apply(ne_bin)

In [11]:
df_test = get_df(u_test)
df_test = df_test[(df_test['ne'] == 'LOC') | (df_test['ne'] == 'LOCORG')]
df_test['ne_bin'] = df_test['ne'].apply(ne_bin)

In [12]:
n = df_dev.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), analyzer='char', tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(df_dev['first_token'])
test_term_doc = vec.transform(df_test['first_token'])

In [13]:
#df_dev['length'] = df_dev['end'] - df_dev['start'] + 1

#df_dev['pos'] = df_dev['first_token'].apply(get_pos)
#df_dev['pos_bin'] = df_dev['pos'].apply(pos_bin)

In [14]:
#df_dev = pd.concat([df_dev, pd.DataFrame(trn_term_doc.toarray())], axis=1)

In [15]:
#df_test['length']= df_test['end'] - df_test['start'] + 1

#df_test['pos'] = df_test['first_token'].apply(get_pos)
#df_test['pos_bin'] = df_test['pos'].apply(pos_bin)

In [16]:
#df_test = pd.concat([df_test, pd.DataFrame(test_term_doc.toarray())], axis=1)

In [17]:
Y_dev = df_dev['ne_bin']
Y_test = df_test['ne_bin']

In [18]:
len(Y_dev), len(Y_test)

(1082, 1574)

In [19]:
#X_dev = df_dev.drop(labels=['tokens', 'ne', 'ne_bin','first_token', 'pos'], axis=1)
#X_test = df_test.drop(labels=['tokens', 'ne', 'ne_bin','first_token', 'pos'], axis=1)

In [20]:
clf = RandomForestClassifier()

In [21]:
clf = MLPClassifier()

In [22]:
clf = xgb.XGBClassifier()

In [23]:
clf.fit(trn_term_doc, Y_dev)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [24]:
predict = clf.predict(test_term_doc)

In [25]:
ans = np.array(Y_test)

In [26]:
a = 0
for i in range(len(predict)):
    if predict[i] == ans[i]:
        a += 1
print(a / len(predict))

0.7115628970775095


In [27]:
predict

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [28]:
u = util.loadAllStandard('./testset/')
for i in range(len(u)):
    res = u[i].makeTokenSets()
    row = []
    for j in range(len(res)):
        token = res[j].toInlineString()
        if token[0] == 'L':
            #Заменить в этом ифе код на код предсказывания
            split = token.split()
            if predict[df_test['tokens_id'] == int(split[1])][0] == 0:
                tag = 'LOC'
            else:
                tag = 'LOCORG'
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (tag, start, end-start+1))
        else:
            split = token.split()
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (split[0], start, end-start+1))
    with open('./result/' + u[i].name + '.task1', 'w') as f:
        f.writelines(row)        

Failed to load the standard of book_3954:
Unknown mention tag: Facility


In [29]:
!python scripts/t1_eval.py -s ./testset -t ./result

Failed to load the standard of book_3954:
Unknown mention tag: Facility
Type    P        R        F1       TP1      TP2      In Std.  In Test.
per        0.9993   0.9993   0.9993  1342.00  1342.00     1343     1343
loc        0.6845   0.6159   0.6484   368.28   368.28      598      538
org        0.9895   0.9895   0.9895  1557.55  1557.55     1574     1574
locorg     0.6415   0.8199   0.7198   519.00   519.00      633      809
overall    0.8879   0.9128   0.9002  3779.83  3779.83     4141     4257
